In [0]:
# cd ./drive/"My Drive"/"Bayes_Stan"/"stan_with_python"
# colab
# !pip3 install arviz

# 階層モデル０１

同一データにおいて、


1. 単純なモデルの検討
2. 階層モデルの検討
3. 階層モデルの動機付け]
4. 階層モデルの特徴

dataはdata-salary-2.txtを使用する。


In [0]:
import os 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

import pystan

from criterion import Criterion
import utils

import arviz
import pickle

from IPython.display import display


plt.style.use("ggplot")

In [0]:
data = pd.read_csv(Path(os.getcwd(), "input", "data-salary-2.txt"))

In [0]:
display(data.head())
display(data.describe())

In [0]:
# 散布図
scatter_data = {}
for i in range(4):
    scatter_data[i] = data.loc[data["KID"]==i+1]
#draw
plt.figure(figsize=(16, 8))
for i in range(4):
    plt.scatter(scatter_data[i]["X"], scatter_data[i]["Y"], label=f"KID : {i}")
plt.legend()
plt.show()

## 単純なモデル

$$Y[n] = y_{base}[n] + \epsilon[n] \tag{1}$$

$$y_{base}[n] = a + b X[n]\tag{2}$$

$$\epsilon[n] \sim Normal(0, \sigma_{Y}) \tag{3}$$

4社にわたる４０人で共通の直線式$a+bX$によって回帰された出力にホワイトノイズを加えたもの

In [0]:
standata = {
    "N":len(data),
    "X":data["X"],
    "Y":data["Y"],
}

In [0]:
# modelを記述した読み込みたいstanファイル
read_stanmodel = "model4-2.stan"
# モデルをセーブしておくpickleファイル
read_file = "model4-2.pkl"

stan_path = Path(os.getcwd(),"model", "stanmodel", read_stanmodel)
pickle_path = Path(os.getcwd(),"model", "model_pkl", read_file)

sm4_2 = utils.read_stanmodel(stan_path, pickle_path)

In [0]:
fit4_2 = sm4_2.sampling(
    data=standata,
    iter=2000,
    warmup=200,
    chains=4,
    seed=496
)

In [0]:
ms = fit4_2.extract()

In [0]:
ms.keys()

In [0]:
arviz.plot_trace(ms);

sigmaの値が大きく推定されている。ここでノイズをグループ差とそれ以外のより小さなノイズに分離して考える。

## 階層モデルの検討

In [0]:
stan_path = Path(os.getcwd(), "model", "stanmodel", "model8-2.stan")
pickle_path = Path(os.getcwd(), "model", "model_pkl", "model8-2.pkl")

sm8_2 = utils.read_stanmodel(stan_path, pickle_path)

In [0]:
standata = {
    "N":len(data),
    "X":data["X"],
    "Y":data["Y"],
    "K":np.max(data["KID"]),
    "KID":data["KID"]
}

In [0]:
fit8_2 = sm8_2.sampling(
    data=standata,
    iter=2000,
    warmup=200,
    chains=4,
    seed=496,
)

In [0]:
print(fit8_2)

In [0]:
ms8_2 = fit8_2.extract()

In [0]:
ms8_2.keys()

In [0]:
arviz.plot_trace(ms8_2)

## 階層モデルの動機付け

## 階層モデルの特徴